In [1]:
import os

from IPython.display import Markdown
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings #langchain_openai.OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain.chat_models import ChatOpenAI ## from langchain_openai import ChatOpenAI

# Load environment variables from .env file
load_dotenv()

# Access API keys from environment variables
my_api_key = os.getenv("OPEN_AI")

## created 2/7/23 

In [3]:
## change to your own save path
def save_db(db_name, input_path, save_path:str=r"C:\Users\zippy\Desktop\Python\Retrieval Augmented Generation\VectorStores\\", chunk_size:int=1250):
    embeddings = OpenAIEmbeddings(api_key=my_api_key)

    ## checks for duplicate
    filenames = os.listdir(save_path)
    for name in filenames:
        if name == (db_name + ".faiss"):
            raise Exception("Name of database already exists in the save directory")

    ## checks file type
    if ".txt" in input_path:
        loader = TextLoader(input_path)
    elif ".pdf" in input_path:
        loader = PyPDFLoader(input_path)
    else:
        raise Exception("Invalid file type, only accepts .txt or .pdf files")
    
    documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=0)
    split_documents = text_splitter.split_documents(documents)
    
    db = FAISS.from_documents(split_documents, embeddings)
    db.save_local(save_path, index_name=db_name)

    return print(f"Saved database at {save_path}")

In [4]:
## change save path here too, they both have to be the same
class dbManager:

    def __init__(self, save_path:str=r"C:\Users\zippy\Desktop\Python\Retrieval Augmented Generation\VectorStores\\"):
        """
        

        """
        self.save_path = save_path
        self.embeddings = OpenAIEmbeddings(api_key=my_api_key)
        self.template = """
        You are a very helpful assistant that answers questions based only on the following context: {context}

        Question: {question}

        You tend to explain concepts in a simple way that is easily understandable.

        Please do not use outside information, only use the information found in the context.
        """

    def load(self, db_name:str):
        db_name = db_name  

        if os.path.exists(self.save_path + db_name + ".faiss"):
            db = FAISS.load_local(self.save_path, self.embeddings, index_name=db_name)
            return db
        else:
            raise FileNotFoundError(f"Vector DB at {self.save_path} does not exist.")

    def test(self, query, db, k_docs=4):
        pass

    def ask(self, query, db, k_docs=4):
        prompt = ChatPromptTemplate.from_template(self.template)

        model = ChatOpenAI(model="gpt-3.5-turbo-0125", openai_api_key=my_api_key, temperature=0.7) #model="gpt-3.5-turbo-0125", 

        output_parser = StrOutputParser()

        retriever = db.as_retriever(search_kwargs={'k': k_docs})

        setup_and_retrieval = RunnableParallel({"context": retriever, "question": RunnablePassthrough()}) 

        chain = setup_and_retrieval | prompt | model | output_parser

        invokemsg = chain.invoke(query)

        return Markdown(invokemsg)

In [5]:
manager = dbManager()

In [6]:
test = manager.load("test")
calculus = manager.load("calculus")
env_slides = manager.load("envslides")
powerlifting = manager.load("powerlifting")
fivethreeone_info = manager.load("531info")
bio44_47 = manager.load("c44-c47_bio")
bio11_20 = manager.load("c11-c20_bio")

In [7]:
manager.ask("""
I have a 22.5 inch torso minus head length, and 17.5 inch femurs. My upper body is 35 inches, and my legs are also 35 inches. My torso without head is relatively longer than my femur length. What is the appropriate squat stance I should take?
""", powerlifting)

c:\Users\zippy\anaconda3\envs\RAG\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


Based on the information provided in the context, with a relatively longer torso compared to your femur length, you would fall under the category of "Short back" and "Long legs" in the table provided. Therefore, the recommended squat stance for you would be with your feet positioned narrow/medium apart and your stance medium/narrow. This stance will help optimize your squatting form and leverage your genetic characteristics for better performance.

In [8]:
manager.ask("""
I have a 22.5 inch torso minus head length (35in with head.), and 35 inch legs. What is the appropriate squat stance I should take?
""", powerlifting)

Based on the information provided in the context, with a torso length of 22.5 inches and leg length of 35 inches, you would fall under the category of "Short back" and "Medium legs" based on the table provided for genetic characteristics for squat stance.

For someone with a short back and medium legs, the recommended squat stance would be medium/wide for both feet, with the toes pointing out slightly.

Therefore, for your body proportions, the appropriate squat stance would be medium/wide with toes pointing out slightly.